## Init e dependencies

In [26]:
! pip install langchain==0.2.7 langchain_aws langchain-community langchain_core boto3 botocore
! pip install PyMuPDF python-docx pydantic

### Init LLM endpoints




In [33]:
import boto3
from langchain_aws import ChatBedrock
from botocore.config import Config
import urllib3
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
import os
from google.colab import userdata
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

bedrock_client = boto3.client(
        service_name='bedrock-runtime',
        region_name='eu-west-1',
        aws_access_key_id=userdata.get('AWS_ACCESS_KEY_ID'),
        aws_secret_access_key=userdata.get('AWS_SECRET_ACCESS_KEY'),
        verify=False,  # Disable SSL verification
        config=Config(
            proxies={'https': None}
        )
    )

llm = ChatBedrock(
        model_id="anthropic.claude-3-haiku-20240307-v1:0",
        client=bedrock_client,
        model_kwargs={
            "temperature": 0,
            "max_tokens": 2000,
        }
    )


### Caricamento Files

In [34]:
import fitz  # PyMuPDF
import docx

from google.colab import files

def upload_file():
  uploaded = files.upload()
  # Una volta eseguita questa cella, Colab ti chiederà di selezionare il file dal tuo PC.

  # 2) Identifica il nome del file caricato
  for file_name in uploaded.keys():
      file_path = file_name

  return file_path

def read_pdf(file_path):
    doc = fitz.open(file_path)
    text = "\n".join([page.get_text("text") for page in doc])
    return text

def read_docx(file_path):
    doc = docx.Document(file_path)
    text = "\n".join([p.text for p in doc.paragraphs])
    return text

def read_txt(file_path):
    with open(file_path, "r") as f:
        text = f.read()
    return text

def upload_and_read_pdf():
    file_path = upload_file()

    return read_pdf(file_path)

def upload_and_read_docx():
    file_path = upload_file()

    return read_docx(file_path)

def upload_and_read_txt():
    file_path = upload_file()

    return read_txt(file_path)

In [ ]:
Summarization

In [37]:
def summarize(text):

    prompt_template = PromptTemplate(
        input_variables=["text"],
        template="""
            Riassumi il seguente testo:
            {text}

            REGOLE
            - scrivi una breve intro per spiegare il contesto
            - riassumi i punti principali in una bullet list
            - trai le tue conclusioni sull'argomento
        """
    )

    chain = prompt_template | llm | StrOutputParser()

    response = chain.invoke({"text": text})
    return response


### Esempio - 1

In [ ]:

text = upload_and_read_pdf()

response = summarize(text)

In [ ]:
from IPython.display import display, Markdown
display(Markdown(f"### Riassunto:\n\n{response}"))


Esempio - 2

In [ ]:
text = upload_and_read_txt()

summary = summarize(text)

display(Markdown(f"### Riassunto:\n\n{response}"))

In [49]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=50000, chunk_overlap=500)
chunks = text_splitter.split_text(text)

print(len(chunks))


12


In [51]:
def summarize_more(text):

    prompt_template = PromptTemplate(
        input_variables=["text"],
        template="""
            Riassumi il seguente testo:
            {text}

            REGOLE
            - per ogni canto ritorna max due righe
        """
    )

    chain = prompt_template | llm | StrOutputParser()

    response = chain.invoke({"text": text})
    return response

summaries = []
for chunk in chunks:
    response = summarize_more(chunk)
    summaries.append(response)


In [ ]:
summary = "\n".join(summaries)
display(Markdown(f"### Riassunto:\n\n{summary}"))

### Esercitazione

Data una query utente in input, ricercare nel testo la risposta e far tornare il testo da cui il modello ha dedotto la risposta